# Stops and Routes Near UW

For Data 511 (Data Viz) Seattle Transportation Group Project 

In [1]:
# import packages
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import time, folium
from folium.plugins import MarkerCluster
from folium import PolyLine
from pyproj import Geod
from geopy.distance import geodesic

# Step 1: Create a Map Centered on UW and an Overlay Bounding Box 

In [2]:
# Create a Map Centered on UW 

#UW lat/long from here: https://www.latlong.net/place/university-of-washington-seattle-wa-usa-12555.html
UW_LAT = 47.655548
UW_LON = -122.303200



UW_MAP = folium.Map(location=[UW_LAT, UW_LON], zoom_start=14)
UW_MAP

In [3]:
#Create a bounding box around UW 

BOX_SCALE = 0.4 # scale in miles 

# translate scale in miles to lat_lon degrees from here: 
# https://www.usgs.gov/faqs/how-much-distance-does-a-degree-minute-and-second-cover-your-maps
lat_scale = float(BOX_SCALE)*(1.0/69.0)    
lon_scale = float(BOX_SCALE)*(1.0/46.54) # cacluated from https://latlongdata.com/distance-calculator/


# calc max/min lat and long for box
min_lat = UW_LAT - lat_scale
max_lat = UW_LAT + lat_scale
min_lon = UW_LON - lon_scale
max_lon = UW_LON + lon_scale


# bounding points
upper_left = (max_lat, min_lon)
upper_right = (max_lat, max_lon) 
lower_right = (min_lat, min_lon)
lower_left = (min_lat, max_lon) 

print(f"Max Lat is_{round(max_lat, 4)} Min Lat is_{round(min_lat, 4)}") 
print(f"Max Lon is_{round(max_lon, 4)} Min Lon is_{round(min_lon, 4)}") 


Max Lat is_47.6613 Min Lat is_47.6498
Max Lon is_-122.2946 Min Lon is_-122.3118


In [4]:
# Draw the bounding box on the map 


# Create a rectangle using the min and max latitude and longitude values
rectangle = folium.Rectangle(
    bounds = [upper_left, upper_right, lower_right, lower_left],
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=.2,
)


# Add the rectangle to the map
rectangle.add_to(UW_MAP)

# Display Map
UW_MAP


# Questions to discuss.

We may want to move the bounding box east to cover more of Udistrict and eless of hte Union Bay Area?  
Should box be bigger, smaller? 

# Step 2: Now let's find the stops near UW

In [5]:
# reprint here the max/min lon/lat of the bounding box for easy reference
print(f"Max Lat is_{round(max_lat, 4)} Min Lat is_{round(min_lat, 4)}") 
print(f"Max Lon is_{round(max_lon, 4)} Min Lon is_{round(min_lon, 4)}") 

Max Lat is_47.6613 Min Lat is_47.6498
Max Lon is_-122.2946 Min Lon is_-122.3118


In [6]:
# load bus and route data cleaned 
path = "C:/Users/suetb/OneDrive - UW/Data511/Group Project/Bus_Route.csv"
routes = pd.read_csv(path)
routes.rename(columns={'X': 'LON'}, inplace=True)
routes.rename(columns={'Y': 'LAT'}, inplace=True)
print(routes.shape)
routes.head()

(9846, 4)


,LON,LAT,STOP_ID,BUS_NUM
0,-122.222391,47.343924,58235,160
1,-122.223371,47.347723,58240,160
2,-122.222411,47.339827,58255,160
3,-122.222390,47.335523,58257,160
4,-122.221679,47.331698,58260,160


In [7]:
# Function that returns true if the stop is in the bounding box based on stop lat/long
# otherwise false

def stop_in_box (stop_lat, stop_lon, min_lat, max_lat, min_lon, max_lon):
    result = (stop_lat >  min_lat) & (stop_lat < max_lat) & (stop_lon > min_lon) & (stop_lon < max_lon)
    return result 


In [8]:
# calculate whether stop in bounding box 

n= routes.shape[0]
near_UW_all = []

for i in range(n):
    stop_lat = routes.iloc[i]["LAT"]
    stop_lon = routes.iloc[i]["LON"]
    near_UW = stop_in_box(stop_lat, stop_lon, min_lat, max_lat, min_lon, max_lon)
    #print(near_UW)
    near_UW_all.append(near_UW)

routes["STOP_IS_NEAR_UW"] = near_UW_all
print()
routes.head()

,LON,LAT,STOP_ID,BUS_NUM,STOP_IS_NEAR_UW
0,-122.222391,47.343924,58235,160,False
1,-122.223371,47.347723,58240,160,False
2,-122.222411,47.339827,58255,160,False
3,-122.222390,47.335523,58257,160,False
4,-122.221679,47.331698,58260,160,False


In [9]:
# how many stops are near UW? 
stops_near_UW = routes[routes["STOP_IS_NEAR_UW"]]
num =len(stops_near_UW["STOP_ID"].drop_duplicates())
print(f"{num}_stops are near UW")
stops_near_UW.head()

26_stops are near UW


,LON,LAT,STOP_ID,BUS_NUM,STOP_IS_NEAR_UW
574,-122.304086,47.650498,25755,255,True
575,-122.303491,47.651546,25765,65,True
1289,-122.298579,47.660850,25790,65,True
1861,-122.310879,47.652085,29240,255,True
1862,-122.306906,47.650504,29242,44N,True


In [10]:
# Function that calculates distance in miles to center of UW - in case we want to change bounding area later 

# ATTRIBUTION NOTE: This function is adapted from code provided by Professor McDonald 
# in a notebook entitled "wildfire_geo_proximity_example.ipbny."  and made available to students
# in Data 512 under a CC-BY 4.0 license 

def calc_point_dist(p1_long, p1_lat, p2_long, p2_lat):
    geodcalc = Geod(ellps='WGS84') 
    distance = geodcalc.inv(p1_long,p1_lat, p2_long, p2_lat)
    d_meters = distance[2]
    d_miles = d_meters * 0.00062137 # constant to convert meters to miles
    return d_miles

In [11]:
n= routes.shape[0]
dist_to_UW_all = []

for i in range(n):
    stop_lat = routes.iloc[i]["LAT"]
    stop_lon = routes.iloc[i]["LON"]
    dist_to_UW = calc_point_dist(stop_lon, stop_lat, UW_LON, UW_LAT)
    #print(dist_to_UW)
    dist_to_UW_all.append(dist_to_UW)

routes["DIST_TO_UW"] = dist_to_UW_all

In [12]:
# Check - If I did this right, the stops selected as "TRUE" by bounding box method
# Should all be within the specified distance to UW * sqrt(2) 
# The sqrt(2) is because I used a box instead of circle - so corner of box further than radial distance
stops_near_UW = routes[routes["STOP_IS_NEAR_UW"]]
stops_near_UW.head()

,LON,LAT,STOP_ID,BUS_NUM,STOP_IS_NEAR_UW,DIST_TO_UW
574,-122.304086,47.650498,25755,255,True,0.351326
575,-122.303491,47.651546,25765,65,True,0.276814
1289,-122.298579,47.660850,25790,65,True,0.425054
1861,-122.310879,47.652085,29240,255,True,0.430978
1862,-122.306906,47.650504,29242,44N,True,0.389037


In [13]:
# Check any stops that seem too far 
max_dist = 2**(1/2)*BOX_SCALE
stops_near_UW[stops_near_UW["DIST_TO_UW"] > max_dist]

,LON,LAT,STOP_ID,BUS_NUM,STOP_IS_NEAR_UW,DIST_TO_UW


All Looks OK. Dataframe empty as expected. 

In [14]:
# Now let's plot stops near UW on a map 

n = stops_near_UW.shape[0]
for i in range (n): 
    dat = stops_near_UW.iloc[i]
    name = dat["STOP_ID"]
    lat = dat["LAT"]
    lon = dat["LON"]
    latlon = (lat, lon)
    folium.Marker(location=latlon, popup=name).add_to(UW_MAP)

UW_MAP

In [15]:
# Save MAP to html file 
UW_MAP.save('UW_map_with_bus_stops.html')


# Step 3: Find all the routes that have stops near UW 

In [27]:
UW_buses = stops_near_UW["BUS_NUM"].drop_duplicates()
num_buses = UW_buses.shape 
bus_names = UW_buses.to_list() 
bus_names_sorted = sorted(bus_names)
print(f"{num_buses[0]} Busses Service UW")
print(f"Bus Routes Servicing UW include_{bus_names_sorted}")

21 Busses Service UW
Bus Routes Servicing UW include_['255', '271', '271N', '31', '32', '372E', '372EN', '43', '44', '44N', '45', '48', '542E', '556E', '65', '67', '73', '75', '982E', '988E', '988EN']


In [17]:
# Add an indicator whether the route has a stop near UW

n= routes.shape[0]
route_has_stop_all = []

for i in range(n): 
    bus_name = routes.iloc[i]['BUS_NUM']
    route_has_stop = bus_name in bus_names
    route_has_stop_all.append(route_has_stop)

routes["ROUTE_SERVES_UW"] = route_has_stop_all
routes.head()

,LON,LAT,STOP_ID,BUS_NUM,STOP_IS_NEAR_UW,DIST_TO_UW,ROUTE_SERVES_UW
0,-122.222391,47.343924,58235,160,False,21.858179,False
1,-122.223371,47.347723,58240,160,False,21.591751,False
2,-122.222411,47.339827,58255,160,False,22.136872,False
3,-122.222390,47.335523,58257,160,False,22.430102,False
4,-122.221679,47.331698,58260,160,False,22.696178,False


In [18]:
# Now filter the route dataset to include only the buses that have  a stop near UW 
routes_filtered = routes[routes["ROUTE_SERVES_UW"]]
routes_filtered = routes_filtered.sort_values(by='BUS_NUM')
print(routes_filtered.shape)
routes_filtered.head()
        

(1084, 7)


,LON,LAT,STOP_ID,BUS_NUM,STOP_IS_NEAR_UW,DIST_TO_UW,ROUTE_SERVES_UW
6084,-122.194316,47.644278,74442,255,False,5.142298,True
6364,-122.185245,47.714881,74734,255,False,6.861687,True
6385,-122.191070,47.711257,74071,255,False,6.494479,True
6386,-122.193585,47.711256,74072,255,False,6.400310,True
2415,-122.313492,47.655921,9575,255,False,0.481116,True


In [19]:
# create clean map w/ no bounding box  
UW_MAP2 = folium.Map(location=[UW_LAT, UW_LON], zoom_start=14)

In [20]:
# Define a function to add a route to the map 

def map_route(this_route):     
    # Add stops to the map with popups
    for index, row in this_route.iterrows():
        folium.Marker([row['LAT'], row['LON']], popup=f"STOP_ID: {row['STOP_ID']}").add_to(UW_MAP2)

    # Connect stops with lines
    #folium.PolyLine(locations=this_route[['Y', 'X']].values, color='blue').add_to(UW_MAP2)

      

In [21]:
# Map a route 

this_bus_name = bus_names[0]
print(this_bus_name)
this_route = routes_filtered[routes_filtered['BUS_NUM'] == this_bus_name]
map_route(this_route)
UW_MAP2



255


In [22]:
# Save MAP to html file 
f = f"Example Route Map for Bus_{this_bus_name}.html"
UW_MAP2.save(f)


In [23]:
# Save filtered routes to CSV 
f = "Bus_Route_Filtered.csv"
routes_filtered.to_csv(f)